In [97]:
import os
from fsl.wrappers import bet
import os
from fsl.wrappers.fslmaths import fslmaths

## Read the Text File with the list of subjects

In [112]:
subjects_file = open('MS_study/subject_list.txt', 'r')
subjects = subjects_file.readlines()
for i in range(len(subjects)):
    subjects[i] = subjects[i].replace('\n', '')

print(subjects)
subjects_file.close()

['subject01_tp1', 'subject01_tp2', 'subject05_tp1', 'subject05_tp2', 'subject05_tp3']


## Managing the Paths to obtain and store files

In [21]:
x = os.getcwdb()
y = str(x).replace("b'", '')
z = y.replace("'", '') + '/MS_study'
print(z)

/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study


In [30]:
path = os.path.join(z, subjects[0])
files = os.listdir(path) # returns list
print(files)

/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp1
['FLAIR_Ax_5mm_20190808173010_7.nii.gz', 'T1_Ax_4mm_TSE_20190808173010_9.nii.gz']


In [44]:
for i in files:
    if 'FLAIR' in i:
        FLAIR = str(path) + '/' + str(i)
    elif 'T1' in i:
        T1 = str(path) + '/' + str(i)

In [43]:
print(FLAIR)
print(T1)

/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp1/FLAIR_Ax_5mm_20190808173010_7.nii.gz
/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp1/T1_Ax_4mm_TSE_20190808173010_9.nii.gz


## Listing other directory path for being fed into the function calls

In [103]:
Flair_Brain = 'MS_study/' + subjects[0] + '/FLAIR_Test_Brain.nii.gz' 
T1_Brain = 'MS_study/' + subjects[0] + '/T1_TP1_Test_Brain'

sample_brain = 'data/Samples/FLAIR_brain_sample.nii.gz'
resampled_FLAIR = 'MS_study/' + subjects[0] + '/resample_FLAIR_TP1_test_to_sample.nii.gz'
MNI_Brain = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'

resampled_FLAIR_To_MNI = 'MS_study/' + subjects[0] + '/resampled_FLAIR_TP1_test_to_sample_TO_MNI.nii.gz'
resampled_FLAIR_To_MNI_mat = 'MS_study/' + subjects[0] + '/resampled_FLAIR_TP1_test_to_sample_TO_MNI.mat'

registration_parameters = '-bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7  -interp nearestneighbour'

T1_Brain_To_Resampled_FLAIR = 'MS_study/' + subjects[0] + '/reg_T1_TP1_test_to_resample_FLAIR_TP1.nii.gz'
T1_Brain_To_Resampled_FLAIR_mat = 'MS_study/' + subjects[0] + '/reg_T1_TP1_test_to_resample_FLAIR_TP1.mat'

FLAIR_Lesion_Mask = 'MS_study/' + subjects[0] + '/FLAIR_Lesion_mask.nii.gz'
Masterfile = 'MS_study/' + subjects[0] + '/masterfile_TP1.txt'

Classifer = 'data/Samples/mytraining'
Bianca_output = 'MS_study/' + subjects[0] + '/bianca_output'
Bianca_output_bin = 'MS_study/' + subjects[0] + '/bianca_output_bin.nii.gz'
Bianca_output_bin_ero_dilM = 'MS_study/' + subjects[0] + '/bianca_output_bin_ero_kernel_box_2_dilM.nii.gz'

## Individual Functions for a specific subjects

In [70]:
def brain_extract():
    bet(FLAIR, Flair_Brain)
    bet(T1,T1_Brain)
brain_extract()

In [89]:
def resample_flair():
    x = os.system(f"flirt -in {Flair_Brain} -ref {sample_brain} -out {resampled_FLAIR} -interp -nearestneighbour -dof 6")
    
resample_flair()

In [ ]:
f"flirt -in FLAIR_Test_Brain.nii.gz -ref FLAIR_brain_sample.nii.gz -out resample_FLAIR_TP1_test_to_sample.nii.gz -applyxfm"

In [92]:
def register_to_MNI():
    x = os.system(
        f"flirt -in {resampled_FLAIR} -ref {MNI_Brain} -out {resampled_FLAIR_To_MNI} -omat {resampled_FLAIR_To_MNI_mat} {registration_parameters}")
register_to_MNI()

In [94]:
def register_t1_to_flair():
    x = os.system(
        f"flirt -in {T1_Brain} -ref {resampled_FLAIR} -out {T1_Brain_To_Resampled_FLAIR} -omat {T1_Brain_To_Resampled_FLAIR_mat} {registration_parameters}")
register_t1_to_flair()

In [98]:
def generate_masterfile():
    file = open(Masterfile, 'w')
    file.write(resampled_FLAIR + ' ')
    file.write(T1_Brain_To_Resampled_FLAIR + ' ')
    file.write(resampled_FLAIR_To_MNI_mat + ' ')
    file.write(FLAIR_Lesion_Mask + ' ')
generate_masterfile()

In [100]:
def run_bianca():
    x = os.system(
        f"bianca --singlefile={Masterfile} --loadclassifierdata={Classifer} --querysubjectnum=1 --brainmaskfeaturenum=1 --featuresubset=1,2 --matfeaturenum=3 -o {Bianca_output} -v")
run_bianca()


In [110]:
def bianca_binary():
    fslmaths(Bianca_output).thr(0.9).bin().run(Bianca_output_bin)
bianca_binary()

In [111]:
def bianca_open():
    x = os.system(f"fslmaths {Bianca_output_bin} -kernel box 2 -ero -dilM {Bianca_output_bin_ero_dilM}")
bianca_open()

# Combining the above code to loop through the all the subjects

In [133]:
subjects_file = open('MS_study/subject_list.txt', 'r')
subjects = subjects_file.readlines()
for i in range(len(subjects)):
    subjects[i] = subjects[i].replace('\n', '')
subjects_file.close()

In [134]:
x = os.getcwdb()
y = str(x).replace("b'", '')
z = y.replace("'", '') + '/MS_study'
print(z)

/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study


## Phase 1 Processing

In [155]:
for i in range(len(subjects)):
    
    path = os.path.join(z, subjects[i])
    files = os.listdir(path) # returns list
    
    for f in files:
        if 'FLAIR' in f:
            FLAIR = str(path) + '/' + str(f)
        elif 'T1' in f:
            T1 = str(path) + '/' + str(f)
    print(files)
    print(FLAIR)
    print(T1)
    print('\n')
    

    Flair_Brain = 'MS_study/' + subjects[i] + '/FLAIR_Test_Brain.nii.gz' 
    T1_Brain = 'MS_study/' + subjects[i] + '/T1_TP1_Test_Brain'

    sample_brain = 'data/Samples/FLAIR_brain_sample.nii.gz'
    resampled_FLAIR = 'MS_study/' + subjects[i] + '/resample_FLAIR_TP1_test_to_sample.nii.gz'
    MNI_Brain = '/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz'

    resampled_FLAIR_To_MNI = 'MS_study/' + subjects[i] + '/resampled_FLAIR_TP1_test_to_sample_TO_MNI.nii.gz'
    resampled_FLAIR_To_MNI_mat = 'MS_study/' + subjects[i] + '/resampled_FLAIR_TP1_test_to_sample_TO_MNI.mat'

    registration_parameters = '-bins 256 -cost normcorr -searchrx -180 180 -searchry -180 180 -searchrz -180 180 -dof 7  -interp nearestneighbour'

    T1_Brain_To_Resampled_FLAIR = 'MS_study/' + subjects[i] + '/reg_T1_TP1_test_to_resample_FLAIR_TP1.nii.gz'
    T1_Brain_To_Resampled_FLAIR_mat = 'MS_study/' + subjects[i] + '/reg_T1_TP1_test_to_resample_FLAIR_TP1.mat'

    FLAIR_Lesion_Mask = 'MS_study/' + subjects[i] + '/FLAIR_Lesion_mask.nii.gz'
    Masterfile = 'MS_study/' + subjects[i] + '/masterfile_TP1.txt'

    Classifer = 'data/Samples/mytraining'
    Bianca_output = 'MS_study/' + subjects[i] + '/bianca_output'
    Bianca_output_bin = 'MS_study/' + subjects[i] + '/bianca_output_bin.nii.gz'
    Bianca_output_bin_ero_dilM = 'MS_study/' + subjects[i] + '/bianca_output_bin_ero_kernel_box_2_dilM.nii.gz'
    
    
    brain_extract()
    resample_flair()
    register_to_MNI()
    register_t1_to_flair()
    generate_masterfile()
    run_bianca()
    bianca_binary()
    bianca_open()

subject01_tp1HEREE
MS_study/subject01_tp1/FLAIR_Test_Brain.nii.gz  HEREE
['FLAIR_Ax_5mm_20190808173010_7.nii.gz', '.DS_Store', 'T1_Ax_4mm_TSE_20190808173010_9.nii.gz']
/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp1/FLAIR_Ax_5mm_20190808173010_7.nii.gz
/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp1/T1_Ax_4mm_TSE_20190808173010_9.nii.gz


subject01_tp2HEREE
MS_study/subject01_tp2/FLAIR_Test_Brain.nii.gz  HEREE
['.DS_Store', 'FLAIR_Ax_5mm_20210121173408_3.nii.gz', 'T1_Ax_4mm_+-_contrast_20210121173408_10.nii.gz']
/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp2/FLAIR_Ax_5mm_20210121173408_3.nii.gz
/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study/subject01_tp2/T1_Ax_4mm_+-_contrast_20210121173408_10.nii.gz


subject05_tp1HEREE
MS_study/subject05_tp1/FLAIR_Test_Brain.nii.gz  HEREE
['FLAIR_Ax_5mm_20190808173010_7.nii.gz', 'T1_Ax_4mm_TSE_20190808173010_9.nii.gz']
/Users/zu

# Phase 2 Processing

## Reading data from the Longitudinal.txt file

In [156]:
long_file = open('MS_study/longitudinal_list.txt', 'r')
longs = long_file.readlines()
for i in range(len(longs)):
    longs[i] = longs[i].replace('\n', '')
long_file.close()

## Obtaining the current working directory to create a path 

In [209]:
print(longs)
x = os.getcwdb()
y = str(x).replace("b'", '')
z = y.replace("'", '')+ '/MS_study'
print(z)

['subject01', 'subject05']
/Users/zunairviqar/Desktop/NYUAD/MRI_Work/Batch_processing/MS_study


## Creating folder to store data for the longitudinal analysis

In [210]:
for i in range(len(longs)):
    path = os.path.join(z, longs[i])
    try:
        os.mkdir(path)
    except OSError as error:
        print(error)

In [163]:
path = os.path.join(z, '')
files = os.listdir(path) # returns list
print(files)

['.DS_Store', 'longitudinal_list.txt', 'subject05_tp3', 'subject05_tp2', 'subject01_tp2', 'subject05_tp1', 'subject01_tp1', 'subject_list.txt']


## Gets the entire list of folders with for the subjects in longitudinal file

In [237]:
folders = []
for i in range(len(longs)):
    folder = []
    for j in range(len(files)):
        if longs[i] in files[j]:
            folder.append(files[j])
    folders.append(folder)
print(folders)

[['subject01_tp2', 'subject01_tp1'], ['subject05_tp3', 'subject05_tp2', 'subject05_tp1']]


## Refines the folders to only focus on the latest and the oldest timepoints

In [238]:
subj_folders = []
for i in range(len(folders)):
    if len(folders[i]) == 2:
        subj_folders.append(folders[i])
    else:
        subj_folders.append([folders[i][0], folders[i][-1]])
print(subj_folders)

[['subject01_tp2', 'subject01_tp1'], ['subject05_tp3', 'subject05_tp1']]


## Combining the above along with the computation

In [236]:
for i in range(len(subj_folders)):    
    Bianca_output_bin_ero_dilM_2 = 'MS_study/' + subj_folders[i][0] + '/bianca_output_bin_ero_kernel_box_2_dilM.nii.gz'
    Bianca_output_bin_ero_dilM_1 = 'MS_study/' + subj_folders[i][1] + '/bianca_output_bin_ero_kernel_box_2_dilM.nii.gz'
    foldr = subj_folders[i][0].split('_')[0]
    
    x = os.system(
        f"fslmaths {Bianca_output_bin_ero_dilM_2} -sub {Bianca_output_bin_ero_dilM_1} MS_study/{foldr}/New_lesion_mask.nii.gz")
    
    x = os.system(
        f"fslmaths {Bianca_output_bin_ero_dilM_2} -add {Bianca_output_bin_ero_dilM_1} MS_study/{foldr}/Total_lesion_mask.nii.gz")
    
    x = os.system(
        f"fslmaths {Bianca_output_bin_ero_dilM} -sub {Bianca_output_bin_ero_dilM_1} MS_study/{foldr}/Old_lesion_mask.nii.gz")
    

['subject01_tp2', 'subject01_tp1']
['subject05_tp3', 'subject05_tp1']
